# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('output/CityWeatherOutputs.csv')
weather_df.drop('Unnamed: 0', axis=1)
# weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,72.55,80,99,24.40,PF,1661722342
1,puerto ayora,-0.7393,-90.3518,76.96,88,95,10.00,EC,1661722342
2,husavik,66.0449,-17.3389,49.12,66,100,1.72,IS,1661722343
3,upernavik,72.7868,-56.1549,38.84,77,43,13.49,GL,1661722344
4,yellowknife,62.4560,-114.3525,57.72,88,100,12.66,CA,1661722255
...,...,...,...,...,...,...,...,...,...
586,nhulunbuy,-12.2333,136.7667,77.61,88,100,9.22,AU,1661722598
587,uhlove,44.8215,33.6045,73.02,70,2,2.01,UA,1661722599
588,northam,51.0333,-4.2167,61.39,74,95,6.08,GB,1661722599
589,mahon,39.8885,4.2658,79.36,69,20,4.61,ES,1661722599


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#configure
gmaps.configure(api_key = g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
lat_lng_locations = weather_df[['Lat','Lng']].astype(float)
humidity = weather_df['Humidity'].astype(float)

#mapping add heatmap layer
layout = {'width': '1000px', 'height': '600px'}
fig = gmaps.figure(map_type = 'HYBRID', layout= layout)

heat_layer = gmaps.heatmap_layer(lat_lng_locations, weights= humidity, max_intensity= 100, point_radius= 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='600px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.

In [4]:
desired_weather_df = weather_df.loc[(weather_df['Max Temp'] <= 80) & (weather_df['Max Temp'] >= 70) &
                                    (weather_df['Wind Speed'] <= 10) & (weather_df['Cloudiness'] < 1)]
desired_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
96,96,nsanje,-16.9200,35.2620,70.68,68,0,4.50,MW,1661722383
190,190,turkistan,43.0000,68.0000,74.32,30,0,9.17,KZ,1661722424
210,210,angoche,-16.2325,39.9086,72.18,74,0,6.17,MZ,1661722433
258,258,gazanjyk,39.2446,55.5154,78.69,18,0,7.52,TM,1661722454
264,264,alanya,36.5438,31.9998,77.13,61,0,3.96,TR,1661722457
300,300,bam,29.1060,58.3570,77.36,7,0,8.63,IR,1661722474
319,319,uruzgan,32.8333,66.0000,70.02,34,0,0.81,AF,1661722482
393,393,saint-pierre,-21.3393,55.4781,71.28,71,0,3.44,RE,1661722514
426,426,pidhorodna,48.1070,30.8895,70.25,37,0,3.80,UA,1661722529
533,533,narimanov,46.6927,47.8498,71.51,35,0,9.60,RU,1661722576


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = desired_weather_df
hotel_df.reset_index(drop= True, inplace= True)
hotel_df.drop('Unnamed: 0', axis=1)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,nsanje,-16.9200,35.2620,70.68,68,0,4.50,MW,1661722383
1,turkistan,43.0000,68.0000,74.32,30,0,9.17,KZ,1661722424
2,angoche,-16.2325,39.9086,72.18,74,0,6.17,MZ,1661722433
3,gazanjyk,39.2446,55.5154,78.69,18,0,7.52,TM,1661722454
4,alanya,36.5438,31.9998,77.13,61,0,3.96,TR,1661722457
5,bam,29.1060,58.3570,77.36,7,0,8.63,IR,1661722474
6,uruzgan,32.8333,66.0000,70.02,34,0,0.81,AF,1661722482
7,saint-pierre,-21.3393,55.4781,71.28,71,0,3.44,RE,1661722514
8,pidhorodna,48.1070,30.8895,70.25,37,0,3.80,UA,1661722529
9,narimanov,46.6927,47.8498,71.51,35,0,9.60,RU,1661722576


In [6]:
#add hotel name column
hotel_df['Hotel Name'] = ' '
                #hotel_df
#link for api info search note to self
# https://developers.google.com/maps/documentation/javascript/places#find_place_from_query
# https://googlemaps.github.io/google-maps-services-python/docs/
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#create parameters
parameters = {"radius": 5000, "rankby": "prominence",
              "types": "lodging", "keyword": "hotel", "key": g_key}

#loop through data frame to grab nearby hotel info using iterrows
for i, j in hotel_df.iterrows():
    

C:\Users\David\AppData\Local\Temp\ipykernel_11068\2723371663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ' '


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map


# Display figure
